# Binary Classification - Titanic Project

### Loading packages and train/dev data.

In [1]:
# Basic packages
import numpy as np # linear algebra.
import pandas as pd # data processing, CSV file I/O.
import matplotlib.pyplot as plt #Graphic tools.
# So we avoid creating popup windows:
%matplotlib inline
import seaborn as sns #Data visualization tools.
# Using a Jupyter dark theme.
!jt -t chesterish -T -N -kl

In [2]:
# In this project, train and test files comes in separate .csv files.

# Loading train data:

# Indicating the filepath from my Github.


file_path_train = "https://raw.githubusercontent.com/jdcimadoro/Binary-Classification-Titanic/main/TitanicData/train.csv"
# Converting it to a Pandas DataFrame.
df_train = pd.read_csv(file_path_train)
# Show first 5 rows to check that it's ok.
# df_train.head()
# Print dataset information.
# df_train.info()

In [3]:
# Loading dev data:

# Indicating the filepath from my Github.
file_path_test = "https://raw.githubusercontent.com/jdcimadoro/Binary-Classification-Titanic/main/TitanicData/test.csv"
# Converting it to a Pandas DataFrame.
df_test = pd.read_csv(file_path_test)
# Show first 5 rows to check that it's ok.
# df_dev.head()
# Print dataset information.
# df_test.info()

### (1) Understand the problem statement, determine the type.
#### In the Titanic project, I want to build a predictive model that answers the question:
#### ¿What sorts of people were more likely to survive?
#### As we will try to predict the "survival" column, being 1 or 0, it is clearly a classification type problem.

In [4]:
# Chequing imbalance of data.
# df_train.Survived.value_counts()
# 0 - 549
# 1 - 342
# It's barely unbalanced, but i will continue thinking it's ok. In a second step, 
# if the models doesn't work properly, i will try resampling techniques.

### (2) Selecting features that I will consider.

In [5]:
# Lets list the default avaiable columns.
print(df_train.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [6]:
# "Name" feature is difficult for processing because it has all unique values.
Name_unique = df_train.Name.unique().size
Name_total = df_train.Name.size
Name_na = df_train.Name.isna().sum()

print("There are %i unique Name values out of %i values, there are %i NA values" % (Name_unique, Name_total, Name_na))
# 891 unique values of 891 values. So I will consider as a hypotesis that "name" is not a critical feature.
# If the model doesn't work properly, "name" could be proccesed using text-based approachs.

There are 891 unique Name values out of 891 values, there are 0 NA values


In [7]:
# "Cabin" feature has a lot of null values and several unique values.

Cabin_unique = df_train.Cabin.unique().size
Cabin_total = df_train.Cabin.size
Cabin_na = df_train.Cabin.isna().sum()

print("There are %i unique Cabin values out of %i values, there are %i NA values" % (Cabin_unique, Cabin_total, Cabin_na))

# So I will consider as a hypotesis that "Cabin" is not a critical feature.
# If the model doesn't work properly, "Cabin" could be imputed, but it is difficult because of the huge proportion of NA values.

There are 148 unique Cabin values out of 891 values, there are 687 NA values


In [8]:
# "Ticket" feature has several repeated values, but it should be unique for each passenger.
# It clearly indicates gross irregularities in the Ticket sell system.
# So I will consider as a hypotesis that "Ticket" feature can be dropped.

Ticket_unique = df_train.Ticket.unique().size
Ticket_total = df_train.Ticket.size
Ticket_na = df_train.Ticket.isna().sum()

print("There are %i unique Ticket values out of %i values, there are %i NA values" % (Ticket_unique, Ticket_total, Ticket_na))

There are 681 unique Ticket values out of 891 values, there are 0 NA values


In [9]:
# "PassengerId" could indicate relationship between passengers, but we have the column "SibSp" (spouse &/or sibling) designed for that.
# So I will consider as a hypotesis that "PassengerId" feature can be dropped.

PassengerId_unique = df_train.PassengerId.unique().size
PassengerId_total = df_train.PassengerId.size
PassengerId_na = df_train.PassengerId.isna().sum()

print("There are %i unique PassengerId values out of %i values, there are %i NA values" % (PassengerId_unique, PassengerId_total, PassengerId_na))

# However, I need the "PassengerId" test column for sending predictions to the leaderboard, so I will store it:
test_PassengerId = df_test.PassengerId

There are 891 unique PassengerId values out of 891 values, there are 0 NA values


In [10]:
# Creating a copy of train and test(dev) DataFrames, and a list with both of them.
df_train_sel = df_train.copy()
df_test_sel = df_test.copy()
all_df = [df_train_sel, df_test_sel]

# Declare the columns to drop.
columns_to_drop = ['Name','Ticket','Cabin','PassengerId']

In [11]:
# Drop the columns.
for dataset in all_df:
    dataset.drop(columns_to_drop,inplace=True,axis=1)

### (3) Thumb rules: Non null values and All Data in Numeric Format

In [12]:
# I will use .value_counts method for identifying trouble values, 
# and the method .isnull().count() for counting null values

In [13]:
# Lets start with "Embarked" feature.

# print("There are %i null values out of %i values in train" 
#      % (df_train_sel.Embarked.isna().sum(),df_train_sel.Embarked.count()))
# print("There are %i null values out of %i values in test" 
#      % (df_test_sel.Embarked.isna().sum(),df_test_sel.Embarked.count()))
# I have only 2 NA values out of 889 in train data, and 0 NA values out of 418 in test data,
# so i will use simple imputation. Lets describe the column to see most frequent values.

# df_train_sel.Embarked.describe()
# df_test_sel.Embarked.describe()

# So I replace the null values in Embarked column with the most frequent value "S".
# Note that we are modifyin both the "train" and the "test(dev)" data.
for dataset in all_df:
    dataset.Embarked.fillna('S')
    
# Once I did this, let's map the values to numbers using a dict:
EmbarkedMap = {'S':0,'C':1,'Q':2}
# Applying the map
for dataset in all_df:
    dataset.Embarked = dataset.Embarked.map(EmbarkedMap) #mapping in three numerical values

In [14]:
# Lets fix "Sex" feature.
[df.Sex.describe() for df in all_df]
# Here, it has 2 unique values, "male" and "female". I will convert this column by using a map.
# Convert this column by using a map.
genderMap = {"male":0,"female":1} #The genderMap is a dict.

# Applying the map
for dataset in all_df:
    dataset.Sex = dataset.Sex.map(genderMap) #Using the map function

In [15]:
# Lets see what is happening with the "Age" feature.
# [df.Age.describe() for df in all_df]
# It has variuos missing values, so imputing using the mean is gross.
# Lets see correlation:
plt.figure(figsize=(4, 5))
heatmap = sns.heatmap(df_train_sel.corr()[['Age']].drop('Age').sort_values(by='Age', ascending=False), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Features Correlating with Age', fontdict={'fontsize':18}, pad=16);

In [16]:
# Also, there exist correlation with other data columns, so I will use KNNImputer in this case.
# I need to remove "Survived" column for using df_train with the model.
# If not, it will stuck with differences in dimension between train and test sets.
imputer_training_columns=df_train_sel.loc[:,['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']]
# Import and declare the model.
from sklearn.impute import KNNImputer
my_imputer = KNNImputer()
# Applying the Imputer.
df_train_imp = pd.DataFrame(my_imputer.fit_transform(X=imputer_training_columns,y=None))
df_test_imp = pd.DataFrame(my_imputer.transform(X=df_test_sel))
# Note that I used "fit_transform" in train data, but "transform" in test data. 
# It is to avoid target leakage.
# Imputation use to remove column names, lets put them back:
df_train_imp.columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']
df_test_imp.columns = df_test_sel.columns
# Add previously dropped "Survived" column from train set.
df_train_imp['Survived']=df_train_sel['Survived']

In [19]:
# Finally, I have cleaned all the data. Check:
df_train_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    float64
 1   Sex       891 non-null    float64
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    float64
 4   Parch     891 non-null    float64
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    float64
 7   Survived  891 non-null    int64  
dtypes: float64(7), int64(1)
memory usage: 55.8 KB


# (4) Selecting Features and Target, select the model

In [43]:
# First, I will select the target column. In this case is "Survived", so lets set X and y according to it.
Xdata = df_train_imp.drop(['Survived'], axis=1)
ydata = df_train_imp.Survived

# I will do the same for the "test" dataframe.
X_test = df_test_imp

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(Xdata,ydata,train_size=0.8,test_size=0.2,random_state=1)

In [78]:
!pip3 install xgboost

In [92]:
#Evaluate several models

#1 LogisticRegression

from sklearn.linear_model import LogisticRegression # Importing the model
clf_lr = LogisticRegression(random_state = 0)
clf_lr.fit(X_train, y_train)

acc_lr = round(clf_lr.score(X_train,y_train)*100, 2) #Calculating the model score.
print("Logistic regression score        %f" %(acc_lr))

#2 Support Vector Machine SVM

from sklearn.svm import SVC
clf_svm = SVC()
clf_svm.fit(X_train, y_train)

acc_svm = round(clf_svm.score(X_train,y_train)*100, 2)
print("Support Vector Machine score     %f" %(acc_svm))

#3 Gaussian Naive Bayes

from sklearn.naive_bayes import GaussianNB
clf_gnb = GaussianNB()
clf_gnb.fit(X_train,y_train)

acc_gnb = round(clf_gnb.score(X_train,y_train)*100,2)
print("Gaussian Naive Bayes score       %f" %(acc_gnb))

#4 Ridge Classifier

from sklearn.linear_model import RidgeClassifier
clf_rc = RidgeClassifier()
clf_rc.fit(X_train, y_train)

acc_rc = round (clf_rc.score(X_train,y_train)*100, 2)
print("Ridge Classifier score           %f" %(acc_rc))

#5 K Nearest Neighbors Classifier (KNN)

from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn.fit(X_train,y_train)

acc_knn = round (clf_knn.score(X_train,y_train)*100, 2)
print("K Nearest Neighbors Classifier   %f" %(acc_knn))

#6 Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf_dt = DecisionTreeClassifier()
clf_dt.fit(X_train,y_train)

acc_dt = round (clf_dt.score(X_train,y_train)*100, 2)
print("Decision Tree Classifier         %f" %(acc_dt))

#7 Random Forest
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train,y_train)

acc_rf = round (clf_dt.score(X_train,y_train)*100, 2)
print("Random Forest Classifier         %f" %(acc_rf))

#8 XGBoost
from xgboost import XGBClassifier
clf_xgb = XGBClassifier(use_label_encoder=False,eval_metric='error')
clf_xgb.fit(X_train,y_train)

acc_xgb = round (clf_xgb.score(X_train,y_train)*100, 2)
print("XGBoost Classifier               %f" %(acc_knn))

Logistic regression score        80.760000
Support Vector Machine score     69.240000
Gaussian Naive Bayes score       79.490000
Ridge Classifier score           80.200000
K Nearest Neighbors Classifier   79.630000
Decision Tree Classifier         98.740000
Random Forest Classifier         98.740000
XGBoost Classifier               79.630000


# (4) Model selection/validation

In [ ]:
# First I have to train the selected model with the full data.
model_selected = RandomForestClassifier()
model_selected.fit(Xdata,ydata)


In [ ]:
# Then, I will send my predictions to the leaderboard.
y_pred = model_selected.predict(X_test)

output = pd.DataFrame({'PassengerId': test_PassengerId, 'Survived': y_pred})
output.to_csv('my_submission.csv', index=False)